## Sequence Edit Distance/Levenshtein Distance

The [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) is an algorithm originally devised to compute the distance between strings by counting the number of character replacements, deletions, and insertions it would take to transform one string into another. Consider the strings 'airplane' and 'inline', for example. The Levenshtein distance between these is 4 because we can delete the first 'a' and 'p' in 'airplane', replace the 'r' with an 'n' and the second 'a' with an 'i' and we obtain 'inline'.

In [1]:
import edist.sed as sed
x = 'airplane'
y = 'inline'
print('The string edit distance between "%s" and "%s" is %d' % (x, y, sed.standard_sed(x, y)))

The string edit distance between "airplane" and "inline" is 4


We can also use backtracing to show which character of 'airplane' has been aligned with which character of 'inline'.

In [2]:
alignment = sed.standard_sed_backtrace(x, y)
print(alignment.render(x, y))

a [0] vs. -
i [1] vs. i [0]
r [2] vs. n [1]
p [3] vs. -
l [4] vs. l [2]
a [5] vs. i [3]
n [6] vs. n [4]
e [7] vs. e [5]


And we can use the 'edits' module to find out which edit operations we have to apply to get from 'airplane' to 'inline'.

In [3]:
import edist.edits as edits
script = edits.alignment_to_script(alignment, x, y)
# note that scripts can only be applied to lists, so we have to convert
# the input string to a list first and then the output list back to a string
x_result = ''.join(script.apply(list(x)))
print('The following edits transform %s to %s: %s' % (x, x_result, str(script)))

The following edits transform airplane to inline: [rep(2, n), rep(5, i), del(3), del(0)]


However, the principle of the sequence edit distance is much more general than that. We can also apply the sequence edit distance to entirely different kinds of sequence data, as long as we are able to define a meaningful distance function between frames. Consider the following two music motifs, for example.

In [4]:
x = [('e', 1. / 16.), ('dsharp', 1. / 16.), ('e', 1. / 16.), ('dsharp', 1. / 16.), ('e', 1. / 16.), ('b', 1. / 16.), ('d', 1. / 16.), ('c', 1. / 16.), ('c', 1. / 16.)]
y = [('gsharp', 1. / 12.), ('csharp', 1. / 12.), ('e', 1. / 12.), ('gsharp', 1. / 12.), ('csharp', 1. / 12.), ('e', 1. / 12.), ('gsharp', 1. / 12.), ('csharp', 1. / 12.), ('e', 1. / 12.), ('gsharp', 1. / 12.), ('csharp', 1. / 12.), ('e', 1. / 12.)]

# define a distance function on notes
notes = ['c', 'csharp', 'd', 'dsharp', 'e', 'f', 'fsharp', 'g', 'gsharp', 'a', 'asharp', 'b']
min_duration = 1. / 32.
max_duration = 1.
def note_delta(x, y):
    if(x is None or y is None):
        return 1.
    x_pitch = notes.index(x[0])
    y_pitch = notes.index(y[0])
    # use the clock distance to get distance between pitches
    pitch_dist = abs(x_pitch - y_pitch)
    pitch_dist = min(pitch_dist, len(notes) - pitch_dist)
    # normalize to get value in the range 0 - 0.5
    pitch_dist /= len(notes)
    # and use the ratio as distance measure between durations
    duration_dist = max(x[1] / y[1], y[1] / x[1])
    # normalize to get value in the range 0 - 0.5
    duration_dist -= 1.
    duration_dist /= 2. * (max_duration / min_duration - 1.)
    # add both values to get a node distance
    return pitch_dist + duration_dist

# compute the edit distance between both motifs
print('The edit distance between the musical motifs is %g' % sed.sed(x, y, note_delta))
# compute the alignment
print('The following notes get aligned')
print(sed.sed_backtrace(x, y, note_delta).render(x, y))

The edit distance between the musical motifs is 4.46505
The following notes get aligned
('e', 0.0625) [0] vs. ('gsharp', 0.08333333333333333) [0]
('dsharp', 0.0625) [1] vs. ('csharp', 0.08333333333333333) [1]
('e', 0.0625) [2] vs. ('e', 0.08333333333333333) [2]
- vs. ('gsharp', 0.08333333333333333) [3]
('dsharp', 0.0625) [3] vs. ('csharp', 0.08333333333333333) [4]
('e', 0.0625) [4] vs. ('e', 0.08333333333333333) [5]
('b', 0.0625) [5] vs. ('gsharp', 0.08333333333333333) [6]
('d', 0.0625) [6] vs. ('csharp', 0.08333333333333333) [7]
('c', 0.0625) [7] vs. ('e', 0.08333333333333333) [8]
- vs. ('gsharp', 0.08333333333333333) [9]
('c', 0.0625) [8] vs. ('csharp', 0.08333333333333333) [10]
- vs. ('e', 0.08333333333333333) [11]


### Affine Edit Distance

In some circumstances, a simple edit distance is not sufficient and we need to use more complicated edit distances. For example, if we compare long texts, we may wish to ensure that locally fitting subsequences are emphasized and not spread over the text. This is supported by the _affine_ edit distance first defined by [Gotoh (1982)](https://doi.org/10.1016/0022-2836(82)90398-9). In this variation, we make long, subsequent deletions/insertions cheaper.

In [5]:
x = 'aaabdb'
y = 'bbccc'

import edist.aed as aed

print('the regular edit distance between the texts is %g' % sed.standard_sed(x, y))
print('the following characters get aligned')
print(sed.standard_sed_backtrace_stochastic(x, y).render(x, y))
print('the affine edit distance betwen the texts is %g' % aed.aed(x, y))
print('the following characters get aligned')
print(aed.aed_backtrace(x, y).render(x, y))

the regular edit distance between the texts is 6
the following characters get aligned
a [0] vs. b [0]
a [1] vs. b [1]
a [2] vs. c [2]
b [3] vs. c [3]
d [4] vs. -
b [5] vs. c [4]
the affine edit distance betwen the texts is 5
the following characters get aligned
del: a [0] vs. -
skdel: a [1] vs. -
skdel: a [2] vs. -
rep: b [3] vs. b [0]
del: d [4] vs. -
rep: b [5] vs. b [1]
ins: - vs. c [2]
skins: - vs. c [3]
skins: - vs. c [4]


### Speedup

To speed up edit distance computations for large data sets, you can use the multiprocess module. Please consult the dtw_demo for more information.